In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlt
from matplotlib import pyplot as plt



In [2]:
df=pd.read_csv("D:/Data Analysis/Capstone project - Personal/data/raw/csv/luke_tech_jobs.csv")
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Data Analyst,"Summer Internship -Data Analyst Intern, Risk M...","Marlborough, MA",via Boatingrevealed.com,"Full-time, Part-time, and Internship",False,"New York, United States",01-01-2024 00:00,False,True,United States,NaN,NaN,NaN,BJ's Wholesale Club,['excel'],{'analyst_tools': ['excel']}
1,Data Analyst,"Staff Data Analyst Operations, Infrastructure ...","Fremont, CA",via ClimateTechList,Full-time,False,"California, United States",01-01-2024 00:00,True,False,United States,NaN,NaN,NaN,Tesla,"['tableau', 'flow']","{'analyst_tools': ['tableau'], 'other': ['flow']}"
2,Data Analyst,Junior Data Analyst - Entry Level,"Waco, TX",via ZipRecruiter,Full-time and Part-time,False,"Texas, United States",01-01-2024 00:00,True,False,United States,NaN,NaN,NaN,Next Recruiting,NaN,NaN
3,Data Analyst,"Data Analyst/Engineer, Supply Chain Optimizati...","Austin, TX",via ClimateTechList,Internship,False,"Texas, United States",01-01-2024 00:00,False,False,United States,NaN,NaN,NaN,Tesla,['spring'],{'libraries': ['spring']}
4,Data Scientist,It analyst,"Tampa, FL",via Talent.com,Full-time,False,"Florida, United States",01-01-2024 00:00,True,False,United States,NaN,NaN,NaN,VirtualVocations,NaN,NaN


### 1. Convert the columns headers to snake case

In [3]:
df.columns=df.columns.str.strip().str.lower().str.replace(" ","_")
df.head()

,job_title_short,job_title,job_location,job_via,job_schedule_type,job_work_from_home,search_location,job_posted_date,job_no_degree_mention,job_health_insurance,job_country,salary_rate,salary_year_avg,salary_hour_avg,company_name,job_skills,job_type_skills
0,Data Analyst,"Summer Internship -Data Analyst Intern, Risk M...","Marlborough, MA",via Boatingrevealed.com,"Full-time, Part-time, and Internship",False,"New York, United States",01-01-2024 00:00,False,True,United States,NaN,NaN,NaN,BJ's Wholesale Club,['excel'],{'analyst_tools': ['excel']}
1,Data Analyst,"Staff Data Analyst Operations, Infrastructure ...","Fremont, CA",via ClimateTechList,Full-time,False,"California, United States",01-01-2024 00:00,True,False,United States,NaN,NaN,NaN,Tesla,"['tableau', 'flow']","{'analyst_tools': ['tableau'], 'other': ['flow']}"
2,Data Analyst,Junior Data Analyst - Entry Level,"Waco, TX",via ZipRecruiter,Full-time and Part-time,False,"Texas, United States",01-01-2024 00:00,True,False,United States,NaN,NaN,NaN,Next Recruiting,NaN,NaN
3,Data Analyst,"Data Analyst/Engineer, Supply Chain Optimizati...","Austin, TX",via ClimateTechList,Internship,False,"Texas, United States",01-01-2024 00:00,False,False,United States,NaN,NaN,NaN,Tesla,['spring'],{'libraries': ['spring']}
4,Data Scientist,It analyst,"Tampa, FL",via Talent.com,Full-time,False,"Florida, United States",01-01-2024 00:00,True,False,United States,NaN,NaN,NaN,VirtualVocations,NaN,NaN


### 2. Drop duplicates

In [4]:
df.drop_duplicates(inplace=True)

### 3. Extract the city from locations

In [5]:
gen_blocklist={
    "remote", "hybrid", "onsite", "on-site", "on site",
    "multiple locations", "anywhere"
}

country_blocklist=set(df['job_country'].dropna().str.lower().str.strip().unique())

def get_city(loc):
    if not isinstance(loc, str):
        return None

    loc_clean=loc.lower().strip()
    city=loc_clean.split(",")[0].strip()

    if(city == "" or city in gen_blocklist or city in country_blocklist or any(ch.isdigit() for ch in city)):
        return None

    return city.title()

df['city']=df['job_location'].apply(get_city)

### 4. Extract the date from job_posted_date

In [6]:
def get_date(date):
    if not isinstance(date, str):
        return None  

    date_str = date.strip()  
    return date_str[:10]     

df['job_date']=df['job_posted_date'].apply(get_date)

### 5. Make the adjusted salary column

In [7]:
def salary_adjusted(row):
    yearly = row["salary_year_avg"]
    hourly = row["salary_hour_avg"]

    if pd.isna(yearly) and pd.isna(hourly):
        return None

    elif pd.notna(yearly):
        return yearly

    elif pd.notna(hourly):
        return hourly * 2080

    else:
        return None

df['salary_adjusted']=df.apply(salary_adjusted, axis=1)

### 6. Clean company names

In [8]:
from rapidfuzz import process, fuzz

extra_set = {
    "inc", "inc.", "llc", "l.l.c.", "co", "co.",
    "corp", "corporation",
    "ltd", "ltd.", "plc", "gmbh",
    "s.a.", "s.a.s",
    "pty", "pty.", "ag"
}

region_words = {
    "amsterdam", "asia", "pacific", "europe", "emea",
    "india", "japan", "singapore", "africa", "america",
    "latam", "apac", "global", "middle", "east",
    "west", "north", "south", "central"
}

country_blocklist = set(
    df['job_country']
    .dropna()
    .str.lower()
    .str.strip()
    .unique()
)

ignore_words = extra_set | region_words | country_blocklist

In [9]:
canonical_brands = [
    "google", "amazon", "microsoft", "meta",
    "ibm", "accenture", "oracle", "uber",
    "dell", "sap", "intel", "cisco",
    "adobe", "salesforce", "paypal"
]

def base_clean_company(name):
    if not isinstance(name, str):
        return None

    n = name.lower().strip()

    for ch in [',','(',')','-','.','_','/']:
        n = n.replace(ch, ' ')

    parts = n.split()

    cleaned = []
    for p in parts:
        if p in ignore_words:
            continue
        cleaned.append(p)

    if not cleaned:
        return None

    return cleaned[0]


In [10]:
def fuzzy_brand_match(clean_name):
    if clean_name is None:
        return None

    best, score, idx = process.extractOne(
        clean_name,
        canonical_brands,
        scorer=fuzz.token_sort_ratio
    )

    if score >= 85:
        return best.title()

    return clean_name.title()

def clean_company(name):
    base = base_clean_company(name)
    final = fuzzy_brand_match(base)
    return final

df["company_clean"] = df["company_name"].apply(clean_company)



### 7. Clean job portals column

In [11]:
ATS_SUFFIXES = {
    "icims", "teamtailor", "jazzhr", "greenhouse", "workday",
    "lever", "freshteam", "smartrecruiters", "bamboohr"
}

def clean_portal(value):
    if not isinstance(value, str):
        return None

    v = value.lower().strip()

    if v.startswith("via"):
        v = v.replace("via", "", 1).strip()

    for sep in ["|", "-", "@"]:
        if sep in v:
            v = v.split(sep)[-1].strip()

    if ".com" in v or ".net" in v or ".org" in v:
        v = v.split(".")[0].strip()

    if all(ch.isdigit() or ch == "." for ch in v):
        return None

    words = v.split()
    cleaned = []

    for w in words:
        if w in ATS_SUFFIXES:
            continue
        if w.isnumeric():
            continue
        if not any(c.isalpha() for c in w):
            continue
        cleaned.append(w)

    if not cleaned:
        return None

    if len(cleaned) >= 2 and cleaned[1] in country_blocklist:
        cleaned = [cleaned[0]]

    return " ".join(cleaned).title()


df["job_portal_clean"] = df["job_via"].apply(clean_portal)


### 8. Explode by skillset

In [12]:
def replace(skillset):
    if not isinstance(skillset, str):
        return None

    str1=skillset.replace('[','').replace("]", "").replace("'", "")

    return str1

df['skillset_str']=df['job_skills'].apply(replace)

In [13]:
df_subset=df[['job_title_short','job_date','salary_adjusted','company_clean','skillset_str']]
df1=df_subset.dropna(subset=['skillset_str'])
df1['skill_list']=df1['skillset_str'].str.split(",")

df_exp=df1.explode('skill_list')

df_exp.head(30)

C:\Users\Asus\AppData\Local\Temp\ipykernel_13360\2500129505.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['skill_list']=df1['skillset_str'].str.split(",")


,job_title_short,job_date,salary_adjusted,company_clean,skillset_str,skill_list
0,Data Analyst,01-01-2024,NaN,Bj'S,excel,excel
1,Data Analyst,01-01-2024,NaN,Tesla,"tableau, flow",tableau
1,Data Analyst,01-01-2024,NaN,Tesla,"tableau, flow",flow
3,Data Analyst,01-01-2024,NaN,Tesla,spring,spring
5,Senior Data Scientist,01-01-2024,NaN,Atlassian,"sql, python, r, c, tableau, microstrategy, sap...",sql
5,Senior Data Scientist,01-01-2024,NaN,Atlassian,"sql, python, r, c, tableau, microstrategy, sap...",python
5,Senior Data Scientist,01-01-2024,NaN,Atlassian,"sql, python, r, c, tableau, microstrategy, sap...",r
5,Senior Data Scientist,01-01-2024,NaN,Atlassian,"sql, python, r, c, tableau, microstrategy, sap...",c
5,Senior Data Scientist,01-01-2024,NaN,Atlassian,"sql, python, r, c, tableau, microstrategy, sap...",tableau
5,Senior Data Scientist,01-01-2024,NaN,Atlassian,"sql, python, r, c, tableau, microstrategy, sap...",microstrategy


In [14]:
df.to_csv("D:\Data Analysis\Capstone project - Personal\data\cleaned\luke_tech_jobs_clean.csv")
df_exp.to_csv("D:\Data Analysis\Capstone project - Personal\data\cleaned\luke_tech_jobs_explodebyskill.csv")

<>:1: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
<>:2: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
<>:1: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
<>:2: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
C:\Users\Asus\AppData\Local\Temp\ipykernel_13360\1993463943.py:1: SyntaxWarning: "\D" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\D"? A raw string is also an option.
  df.to_csv("D:\Data Analysis\Capstone project - Personal\data\cleaned\luke_tech_jobs_clean.csv")
C:\Users\Asus\AppData\Local\Temp\ipykernel_13360\1993463943.py:2: SyntaxWarni